In [1]:
import pandas as pd
import json

In [2]:
from raw_fetch import download_play_by_play, download_players, download_snap_counts
from pbp_raw_to_play_player import transform_pbp_raw_to_play_player
from play_player_to_game_player import transform_play_player_to_game_player
from t_game_player_to_season_player import transform_game_player_to_season_player
from fantasy import calculate_fantasy_points
from player import merge_player_metadata
from player_stats_enhanced import game_player_stats

In [3]:
fantasy_config = json.load(open('fantasy_config.json'))

In [4]:
player_df = pd.read_parquet('./raw/players/players.parquet')

In [5]:
play_by_play = pd.read_parquet('./raw/pbp/play_by_play_2024.parquet')

In [6]:
pbp = pd.read_parquet('./raw/pbp/play_by_play_2024.parquet')

In [7]:
game_play_stats = pd.read_parquet(
    './raw/player_stats/player_stats_2024.parquet')

In [8]:
game_play_stats.columns

Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points

In [9]:
en_game_play_stats = game_player_stats(game_play_stats, fantasy_config)

In [13]:
en_game_play_stats[game_play_stats['week']
                   == 4].sort_values(by='pos_rank').head(20)

,player_id,player_name,player_display_name,position,position_group,headshot_url,pos_team,season,week,season_type,...,fantasy_points,fantasy_points_ppr,pass_fp,rush_fp,rec_fp,fp,rush_share,pass_share,rank,pos_rank
609,00-0036554,N.Collins,Nico Collins,WR,WR,https://static.www.nfl.com/image/upload/f_auto...,HOU,2024,4,REG,...,21.10,33.10,0.00,0.0,27.1,27.10,0.000000,0.000000,6.0,1.0
114,00-0032764,D.Henry,Derrick Henry,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,BAL,2024,4,REG,...,32.90,35.90,0.00,25.9,8.5,34.40,0.705882,0.000000,1.0,1.0
1026,00-0038996,T.Kraft,Tucker Kraft,TE,TE,https://static.www.nfl.com/image/upload/f_auto...,GB,2024,4,REG,...,11.30,17.30,0.00,0.0,14.3,14.30,0.000000,0.000000,56.5,1.0
677,00-0036945,J.Fields,Justin Fields,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,PIT,2024,4,REG,...,31.98,31.98,14.48,17.5,0.0,31.98,0.333333,1.000000,2.0,1.0
14,00-0028872,J.Hekker,Johnny Hekker,P,SPEC,https://static.www.nfl.com/image/upload/f_auto...,CAR,2024,4,REG,...,0.00,0.00,0.00,0.0,0.0,0.00,0.000000,0.024390,283.5,1.0
22,00-0029892,K.Juszczyk,Kyle Juszczyk,FB,RB,https://static.www.nfl.com/image/upload/f_auto...,SF,2024,4,REG,...,1.20,2.20,0.00,0.0,1.7,1.70,0.000000,0.000000,228.0,1.0
1080,00-0039146,J.Reed,Jayden Reed,WR,WR,https://static.www.nfl.com/image/upload/f_auto...,GB,2024,4,REG,...,20.10,27.10,0.00,0.2,23.4,23.60,0.052632,0.000000,11.0,2.0
657,00-0036894,P.Freiermuth,Pat Freiermuth,TE,TE,https://static.www.nfl.com/image/upload/f_auto...,PIT,2024,4,REG,...,11.70,16.70,0.00,0.0,14.2,14.20,0.000000,0.000000,58.0,2.0
896,00-0038134,K.Walker,Kenneth Walker,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,SEA,2024,4,REG,...,29.60,33.60,0.00,26.0,5.6,31.60,0.631579,0.000000,3.0,2.0
359,00-0034855,B.Mayfield,Baker Mayfield,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,TB,2024,4,REG,...,28.88,28.88,21.88,7.0,0.0,28.88,0.160000,1.000000,4.0,2.0


In [ ]:
en_game_play_stats[game_play_stats['week'] == 4].sort_values(by='pos_rank')

In [17]:
pbp[pbp['fumble'] > 0]

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
20,528.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,BUF,home,ARI,...,0.0,0.0,-6.044650,NaN,NaN,NaN,NaN,NaN,0.478197,52.180350
99,2527.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,-4.927136,NaN,NaN,NaN,NaN,NaN,0.691430,30.856955
199,921.0,2024_01_BAL_KC,2024090500,KC,BAL,REG,1,BAL,away,KC,...,0.0,0.0,-5.576134,NaN,NaN,NaN,NaN,NaN,0.457668,54.233199
396,1413.0,2024_01_CAR_NO,2024090806,NO,CAR,REG,1,CAR,away,NO,...,0.0,1.0,-4.797649,NaN,NaN,NaN,NaN,NaN,0.680536,-68.053603
505,4250.0,2024_01_CAR_NO,2024090806,NO,CAR,REG,1,CAR,away,NO,...,0.0,1.0,3.035296,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7743,1239.0,2024_03_PHI_NO,2024092203,NO,PHI,REG,3,PHI,away,NO,...,0.0,1.0,-5.586196,NaN,NaN,NaN,NaN,NaN,0.617259,38.274115
7823,3253.0,2024_03_PHI_NO,2024092203,NO,PHI,REG,3,PHI,away,NO,...,0.0,1.0,2.814311,1.725931,4.812842,3.0,0.634611,0.634611,0.970203,2.979702
7933,1965.0,2024_03_SF_LA,2024092211,LA,SF,REG,3,SF,away,LA,...,0.0,0.0,-3.085736,NaN,NaN,NaN,NaN,NaN,0.967370,3.262955
8027,4434.0,2024_03_SF_LA,2024092211,LA,SF,REG,3,SF,away,LA,...,0.0,0.0,-0.071247,0.393172,3.694635,1.0,1.000000,1.000000,0.973545,2.645463


In [18]:
play_player_raw = transform_pbp_raw_to_play_player(pbp)

In [19]:
play_player_fantasy = calculate_fantasy_points(fantasy_config, play_player_raw)

In [20]:
play_player = merge_player_metadata(
    play_player_fantasy, player_df)

In [21]:
game_player = transform_play_player_to_game_player(play_player)

In [22]:
season_player = transform_game_player_to_season_player(game_player)

## QC Checks


## Save Data


In [13]:
play_player.to_parquet('./tables/play_player_2024.parquet')

In [16]:
game_player.to_parquet('./tables/game_player_2024.parquet')

In [23]:
season_player.to_parquet('./tables/season_player_2024.parquet')

In [25]:
# save to frontend
play_player.to_parquet('../src/data/play_player_2024.parquet', index=False)
game_player.to_parquet('../src/data/game_player_2024.parquet', index=False)
season_player.to_parquet('../src/data/season_player_2024.parquet', index=False)